In [0]:
base_path='/Volumes/workspace/synthea/synthea_datasets'
files=dbutils.fs.ls(base_path) #list files in volume
#files

In [0]:
for file in files:
    file_path = file.path

    # Skip folders and non-data files
    if file.isDir():
        continue

    # CSV to Delta
    if file_path.lower().endswith(".csv"):
        print(f"Converting CSV: {file_path}")
        df = spark.read.csv(file_path, header=True, inferSchema=True)
        delta_path = file_path.replace(".csv", "_delta")
        df.write.format("delta").mode("overwrite").save(delta_path)

    # JSON to Delta
    elif file_path.lower().endswith(".json"):
        print(f"Converting JSON: {file_path}")
        df = spark.read.json(file_path)
        delta_path = file_path.replace(".json", "_delta")
        df.write.format("delta").mode("overwrite").save(delta_path)

In [0]:
supplies_df = spark.read.format("delta").load(
    '/Volumes/workspace/synthea/synthea_datasets/supplies_delta'
)

# Display sample
display(supplies_df.limit(5))

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/synthea/synthea_datasets/patients_delta`
LIMIT 5


In [0]:
base_path = "/Volumes/workspace/synthea/synthea_datasets"
db_name = "synthea"


spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")
for item in dbutils.fs.ls(base_path):
    if item.isDir() and item.name.lower().endswith("_delta/"):
        view_name = item.name[:-7]  # remove "_delta"
        delta_path = item.path.rstrip("/")
        print(f"Creating view: {db_name}.{view_name}")
        spark.sql(f"CREATE OR REPLACE VIEW {db_name}.{view_name} AS SELECT * FROM delta.`{delta_path}`")



In [0]:
%sql
SELECT id,city FROM synthea.patients